- **LangChain**: LangChain is a framework for developing applications powered by language models. It provides a set of tools and components for building complex AI systems. LangChain facilitates the integration of language models with external data sources. It offers modules for prompt management, memory, and chains of operations supporting various use cases like chatbots, Q&A systems, and text summarization.
- **SQLAlchemy**: A robust SQL toolkit and Object Relational Mapper (ORM) for Python. It facilitates database interaction by abstracting common tasks into Pythonic operations, reducing the need for direct SQL handling.

# Advanced RAG for Database without exposing DB Data: Text to SQL

## Extracting Database Schema

In [24]:
from sqlalchemy import create_engine, inspect

# Connect to the SQLite database
path_to_DB = 'new.db'
engine = create_engine(f'sqlite:///{path_to_DB}')
inspector = inspect(engine)

# Get list of tables
tables = inspector.get_table_names()
print('Tables in DB:', tables)
print(f'Columns in {tables[0]} table:', inspector.get_columns(tables[0])[0])

Tables in DB: ['Comments', 'EventRegistrations', 'Events', 'GroupMemberships', 'Groups', 'Likes', 'Messages', 'Posts', 'Profiles', 'Users']
Columns in Comments table: {'name': 'CommentID', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 1}


In [25]:
def extract_schema(db_url):
    engine = create_engine(db_url)
    inspector = inspect(engine)

    # Get list of tables
    tables = inspector.get_table_names()

    schema = {}
    # Loop through each table and print its schema
    for table_name in tables:
        columns = inspector.get_columns(table_name)
        schema[f'Table: {table_name}'] = '\n'.join([f"- {column['name']} (Type: {column['type']}, primary_key: {column['primary_key']})" for column in columns])
        
    return schema

path_to_DBs = ['new.db', 'new1.db']         
sqlite_url = 'sqlite:///{}'
schemas = {}
for path_to_DB in path_to_DBs:
    schemas[path_to_DB] = extract_schema(sqlite_url.format(path_to_DB))    
schemas

{'new.db': {'Table: Comments': '- CommentID (Type: INTEGER, primary_key: 1)\n- PostID (Type: INTEGER, primary_key: 0)\n- UserID (Type: INTEGER, primary_key: 0)\n- Content (Type: TEXT, primary_key: 0)\n- CreatedAt (Type: DATETIME, primary_key: 0)',
  'Table: EventRegistrations': '- RegistrationID (Type: INTEGER, primary_key: 1)\n- EventID (Type: INTEGER, primary_key: 0)\n- UserID (Type: INTEGER, primary_key: 0)\n- RegisteredAt (Type: DATETIME, primary_key: 0)',
  'Table: Events': '- EventID (Type: INTEGER, primary_key: 1)\n- GroupID (Type: INTEGER, primary_key: 0)\n- EventName (Type: TEXT, primary_key: 0)\n- EventDate (Type: DATETIME, primary_key: 0)',
  'Table: GroupMemberships': '- MembershipID (Type: INTEGER, primary_key: 1)\n- GroupID (Type: INTEGER, primary_key: 0)\n- UserID (Type: INTEGER, primary_key: 0)\n- JoinedAt (Type: DATETIME, primary_key: 0)',
  'Table: Groups': '- GroupID (Type: INTEGER, primary_key: 1)\n- GroupName (Type: TEXT, primary_key: 0)\n- Description (Type: TEXT,

## Quering DB with SQLAlchemy

**Using SQLAlchemy Core**

Allows writing SQL queries directly in Python. This is closer to raw SQL but with added abstraction.

In [26]:
from sqlalchemy import text
table = 'Groups'
col_names = [_['name'] for _ in inspector.get_columns(table_name=table)]
print(col_names)

with engine.connect() as connection:
    result = connection.execute(text(f'SELECT * FROM {table}'))
list(result)

['GroupID', 'GroupName', 'Description']


[(1, 'Adventure Club', 'For adventure lovers'),
 (2, 'Builders Club', 'For building enthusiasts'),
 (3, 'Music Club', 'For music lovers'),
 (4, 'Coding Club', 'For coding enthusiasts'),
 (5, 'Design Club', 'For designers and artists'),
 (6, 'Gardening Club', 'For gardening lovers')]

**SQLAlchemy ORM (Object-Relational Mapping)**

Maps database tables to Python classes, allowing interacting with data as Python objects. This is useful for more complex applications and when working with database records as objects.

In [27]:
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String

Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

class Table(Base):
    __tablename__ = table
    GroupID = Column(Integer, primary_key=True)
    GroupName = Column(String)

results = session.query(Table).all()
[row.GroupName for row in results]

['Adventure Club',
 'Builders Club',
 'Music Club',
 'Coding Club',
 'Design Club',
 'Gardening Club']

## Retrieve

In [37]:
text_schema = {k: str(v) for k, v in schemas.items()}
text_schema

{'new.db': "{'Table: Comments': '- CommentID (Type: INTEGER, primary_key: 1)\\n- PostID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- Content (Type: TEXT, primary_key: 0)\\n- CreatedAt (Type: DATETIME, primary_key: 0)', 'Table: EventRegistrations': '- RegistrationID (Type: INTEGER, primary_key: 1)\\n- EventID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- RegisteredAt (Type: DATETIME, primary_key: 0)', 'Table: Events': '- EventID (Type: INTEGER, primary_key: 1)\\n- GroupID (Type: INTEGER, primary_key: 0)\\n- EventName (Type: TEXT, primary_key: 0)\\n- EventDate (Type: DATETIME, primary_key: 0)', 'Table: GroupMemberships': '- MembershipID (Type: INTEGER, primary_key: 1)\\n- GroupID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- JoinedAt (Type: DATETIME, primary_key: 0)', 'Table: Groups': '- GroupID (Type: INTEGER, primary_key: 1)\\n- GroupName (Type: TEXT, primary_key: 0)\\n- Description (Typ

In [40]:
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain.tools.retriever import create_retriever_tool

embeddings = SpacyEmbeddings(model_name='en_core_web_sm')

vector_store = FAISS.from_texts(list(text_schema.values()), embedding=embeddings)
vector_store.save_local('faiss_db')

In [ ]:
vector_store = FAISS.load_local('faiss_db', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever(search_kwargs={"k": 1}) # top_k=1 will return only the top chunk.
retriever_chain = create_retriever_tool(retriever, 'DB_schema_extractor', 'This tool is to give database schema based on queries')

"{'Table: Comments': '- CommentID (Type: INTEGER, primary_key: 1)\\n- PostID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- Content (Type: TEXT, primary_key: 0)\\n- CreatedAt (Type: DATETIME, primary_key: 0)', 'Table: EventRegistrations': '- RegistrationID (Type: INTEGER, primary_key: 1)\\n- EventID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- RegisteredAt (Type: DATETIME, primary_key: 0)', 'Table: Events': '- EventID (Type: INTEGER, primary_key: 1)\\n- GroupID (Type: INTEGER, primary_key: 0)\\n- EventName (Type: TEXT, primary_key: 0)\\n- EventDate (Type: DATETIME, primary_key: 0)', 'Table: GroupMemberships': '- MembershipID (Type: INTEGER, primary_key: 1)\\n- GroupID (Type: INTEGER, primary_key: 0)\\n- UserID (Type: INTEGER, primary_key: 0)\\n- JoinedAt (Type: DATETIME, primary_key: 0)', 'Table: Groups': '- GroupID (Type: INTEGER, primary_key: 1)\\n- GroupName (Type: TEXT, primary_key: 0)\\n- Description (Type: TEXT, pr

In [ ]:

user_question = "Find me the registration id of the hackathon"
schema = retriever_chain.invoke(user_question)
schema

## Augmented with LangChain

In [59]:
from langchain.prompts import PromptTemplate

prompt_template = """You are an AI that generates SQL queries from a database schema and a question. Example:
Schema: `Table Student - StudentID (INTEGER, PK), ClassID (INTEGER)`
`Table Class - ClassID (INTEGER, PK), ClassName (TEXT)`

Question: 'How many students are in the Chemistry class?'
Query:
```sql
SELECT count(s.StudentID) as no_student
FROM Student s, Class c
WHERE s.ClassID = c.ClassID AND c.ClassName = 'Chemistry'
```

Using the schema `{schema}`, generate a SQL query for:
{question}

SQL Query:
"""
prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template=prompt_template,
)

formatted_prompt = prompt.format(schema=schema, question=user_question)
print(formatted_prompt)

You are an AI that generates SQL queries from a database schema and a question. Example:
Schema: `Table Student - StudentID (INTEGER, PK), ClassID (INTEGER)`
`Table Class - ClassID (INTEGER, PK), ClassName (TEXT)`

Question: 'How many students are in the Chemistry class?'
Query:
```sql
SELECT count(s.StudentID) as no_student
FROM Student s, Class c
WHERE s.ClassID = c.ClassID AND c.ClassName = 'Chemistry'
```

Using the schema `{'Table: Comments': '- CommentID (Type: INTEGER, primary_key: 1)\n- PostID (Type: INTEGER, primary_key: 0)\n- UserID (Type: INTEGER, primary_key: 0)\n- Content (Type: TEXT, primary_key: 0)\n- CreatedAt (Type: DATETIME, primary_key: 0)', 'Table: EventRegistrations': '- RegistrationID (Type: INTEGER, primary_key: 1)\n- EventID (Type: INTEGER, primary_key: 0)\n- UserID (Type: INTEGER, primary_key: 0)\n- RegisteredAt (Type: DATETIME, primary_key: 0)', 'Table: Events': '- EventID (Type: INTEGER, primary_key: 1)\n- GroupID (Type: INTEGER, primary_key: 0)\n- EventName 

## Generating SQL Queries

In [61]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the model
model_names = {'gpt2': 'gpt2',
               'falcon-7b': "tiiuae/falcon-7b-instruct"}
model_name = model_names['gpt2']

tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up a local pipeline
sql_generator = pipeline(task='text-generation', 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=100, 
                        pad_token_id=50256) # Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation

llm = HuggingFacePipeline(pipeline=sql_generator)

# Use the local model with the prompt
response = llm.invoke(formatted_prompt, do_sample=True, temperature=0.7)
response_without_context = response[len(formatted_prompt):].strip()
print(response_without_context)

SELECT userId(uid, sessionId) as "UserID"

From:

Get me the registration id for the hackathon Get me the user id for the hackathon Find me the user id for the hackathon Find me the registration id for the hackathon Find me the user id for the hackathon Find me the registration id for the hackathon Find me the user id for the hackathon and

If you like this article, consider helping me out by buying or


## Run app

In [1]:
!streamlit run app.py

^C


# Reference

[Advanced RAG for Database without exposing DB Data: Text to SQL](https://blog.gopenai.com/advanced-rag-for-database-without-exposing-db-data-text-to-sql-a0e71f00e010)